<a href="https://colab.research.google.com/github/Amisha1019/Emotion-Detection-Internship-/blob/main/Task_6_of_emotion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install streamlit torch torchvision pillow opencv-python-headless numpy pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 66.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 64.1 MB/s eta 0:00:00


In [4]:
!pip install streamlit

In [20]:
code = r"""
import streamlit as st
import torch
import torchvision
from torchvision import transforms
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import io
import os
from datetime import datetime
import pytz

# --- Configuration Constants ---
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_CONFIDENCE_THRESHOLD = 0.5
TIMEZONE = 'Asia/Kolkata'

COCO_INSTANCE_CATEGORY_NAMES = [
    '__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck',
    'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
    'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
    'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
    'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
    'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
    'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
    'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
    'teddy bear', 'hair drier', 'toothbrush'
]
CAR_CLASS_NAMES = {'car', 'truck', 'bus', 'motorcycle'}

# --- Global DataFrames for logging (used in session_state) ---
image_log_columns = ['timestamp', 'filename', 'num_vehicles', 'vehicle_details', 'num_people']
video_log_columns = ['timestamp', 'frame_index', 'num_vehicles', 'vehicle_details', 'num_people']

# --- Model Loading ---
@st.cache_resource
def load_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    model.to(DEVICE)
    return model

model = load_model()

# --- Utility Functions ---
def get_current_time_iso():
    tz = pytz.timezone(TIMEZONE)
    return datetime.now(tz).isoformat()

def detect_objects_pytorch(model, image: np.ndarray):
    # image is BGR (OpenCV). Convert to RGB PIL then to tensor
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(img_rgb)
    transform = transforms.Compose([transforms.ToTensor()])
    tensor = transform(pil).to(DEVICE)
    with torch.no_grad():
        outputs = model([tensor])
    return outputs[0]

def bbox_to_int(bbox):
    return [int(b) for b in bbox]

def crop_region(image, bbox):
    x1, y1, x2, y2 = bbox_to_int(bbox)
    h, w = image.shape[:2]
    x1 = max(0, x1); y1 = max(0, y1); x2 = min(w-1, x2); y2 = min(h-1, y2)
    return image[y1:y2, x1:x2]

def detect_dominant_color(bgr_image):
    # Convert to HSV and compute mean hue over non-black pixels
    if bgr_image.size == 0:
        return 'unknown', None
    hsv = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    # Create mask for non-dark pixels
    v = hsv[:, :, 2]
    mask = v > 30
    if np.count_nonzero(mask) < 10:
        return 'unknown', None
    hues = hsv[:, :, 0][mask]
    mean_hue = np.mean(hues)
    # Hue in OpenCV: 0-179 (0 red, 60 yellow, 120 blue)
    # We'll classify blue if hue in range ~[90,130]
    if 90 <= mean_hue <= 130:
        return 'blue', mean_hue
    # simple other buckets (red, green, yellow) could be added; otherwise 'other'
    return 'other', mean_hue

def draw_bounding_boxes(frame, detections, labels_of_interest, conf_thresh=MODEL_CONFIDENCE_THRESHOLD):
    boxes = detections['boxes'].cpu().numpy()
    scores = detections['scores'].cpu().numpy()
    labels = detections['labels'].cpu().numpy()

    vehicles = []
    people_count = 0
    # ensure frame has shape for h, w access
    if frame is not None and frame.shape:
        h, w = frame.shape[:2]
    else:
        h, w = 0, 0 # Default if frame is empty or malformed

    for bbox, score, label_idx in zip(boxes, scores, labels):
        if score < conf_thresh:
            continue
        label_name = COCO_INSTANCE_CATEGORY_NAMES[label_idx]
        if label_name == 'person':
            people_count += 1
        if label_name in labels_of_interest:
            # Handle vehicle
            crop = crop_region(frame, bbox)
            color_label, hue = detect_dominant_color(crop)
            # According to requirement:
            # - show red rectangle for blue cars
            # - show blue rectangles for other colour cars
            if color_label == 'blue':
                rect_color = (0, 0, 255) # BGR red
            elif color_label == 'unknown':
                rect_color = (255, 0, 0) # blue as fallback
            else:
                rect_color = (255, 0, 0) # BGR blue
            x1, y1, x2, y2 = bbox_to_int(bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), rect_color, 2)
            label_text = f"{label_name} ({color_label}) {score:.2f}"
            cv2.putText(frame, label_text, (x1, max(15, y1-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, rect_color, 1)
            vehicles.append({'bbox': bbox_to_int(bbox), 'score': float(score), 'label': label_name, 'color': color_label})

    return frame, vehicles, people_count

# --- Streamlit UI ---
st.set_page_config(page_title='Car Colour Detector', layout='wide')
st.title('Car Colour Detection & People Counter')

# Session state initialization
if 'image_log' not in st.session_state:
    st.session_state['image_log'] = pd.DataFrame(columns=image_log_columns)
if 'video_log' not in st.session_state:
    st.session_state['video_log'] = pd.DataFrame(columns=video_log_columns)

col1, col2 = st.columns([1, 2])

with col1:
    st.header('Settings')
    st.write('Model: torchvision Faster R-CNN (pretrained on COCO)')
    st.write('Vehicle classes considered: car, truck, bus, motorcycle')
    st.write(f'Detection confidence threshold: {MODEL_CONFIDENCE_THRESHOLD}')
    st.markdown('---')
    st.header('Logs')
    if st.button('Download image log (.csv)'):
        csv = st.session_state['image_log'].to_csv(index=False)
        st.download_button('Download image log', csv, file_name='image_log.csv')
    if st.button('Download video log (.csv)'):
        csv = st.session_state['video_log'].to_csv(index=False)
        st.download_button('Download video log', csv, file_name='video_log.csv')

with col2:
    st.header('Upload an image')
    uploaded_file = st.file_uploader('Choose an image', type=['png', 'jpg', 'jpeg'])
    if uploaded_file is not None:
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
        st.image(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), caption='Uploaded image', use_column_width=True)
        with st.spinner('Detecting...'):
            detections = detect_objects_pytorch(model, img)
            out_frame, vehicles, people_count = draw_bounding_boxes(img.copy(), detections, CAR_CLASS_NAMES)
            st.image(cv2.cvtColor(out_frame, cv2.COLOR_BGR2RGB), caption='Detections', use_column_width=True)
        # Log results
        new_image_log_entry = pd.DataFrame([{
            'timestamp': get_current_time_iso(),
            'filename': uploaded_file.name,
            'num_vehicles': len(vehicles),
            'vehicle_details': vehicles,
            'num_people': people_count
        }], columns=image_log_columns)
        st.session_state['image_log'] = pd.concat([st.session_state['image_log'], new_image_log_entry], ignore_index=True)
        st.success(f'Found {len(vehicles)} vehicles and {people_count} people.')

    st.markdown('---')
    st.header('Real-time webcam (press Start)')
    start_webcam = st.checkbox('Start webcam')
    conf_thresh = st.slider('Confidence threshold', 0.1, 0.9, 0.5)

    if start_webcam:
        FRAME_WINDOW = st.image([])
        cap = cv2.VideoCapture(0)
        frame_idx = 0
        if not cap.isOpened():
            st.error('Webcam not available')
        else:
            try:
                while start_webcam: # This loop will run as long as the checkbox is ticked
                    ret, frame = cap.read()
                    if not ret:
                        st.error('Failed to read from webcam')
                        break
                    detections = detect_objects_pytorch(model, frame)
                    out_frame, vehicles, people_count = draw_bounding_boxes(frame.copy(), detections, CAR_CLASS_NAMES, conf_thresh)
                    FRAME_WINDOW.image(cv2.cvtColor(out_frame, cv2.COLOR_BGR2RGB))
                    # Log every Nth frame to avoid huge logs
                    if frame_idx % 10 == 0 and (len(vehicles) > 0 or people_count > 0):
                        new_video_log_entry = pd.DataFrame([{
                            'timestamp': get_current_time_iso(),
                            'frame_index': frame_idx,
                            'num_vehicles': len(vehicles),
                            'vehicle_details': vehicles,
                            'num_people': people_count
                        }], columns=video_log_columns)
                        st.session_state['video_log'] = pd.concat([st.session_state['video_log'], new_video_log_entry], ignore_index=True)
                    frame_idx += 1
                    # Streamlit reruns the script on widget interaction.
                    # No need to re-check inside the loop. The loop will naturally break if user unchecks and script reruns.
            finally:
                cap.release()

st.markdown('---')
st.write('Notes and tips:')
st.write('- For better car-color detection, use close, high-resolution images of vehicles.')
st.write('- The colour heuristic uses mean hue and is intentionally simple: you can replace it with k-means on the crop for improved robustness.')

st.write('\nIf you want, I can:')
st.write('- Provide a Colab-friendly version (with ngrok) so you can run the Streamlit UI from Colab.')
st.write('- Replace the simple colour heuristic with K-means clustering for dominant colour extraction.')
st.write('- Save detected car crop images alongside logs (zipped) for manual review.')

# End of file
"""

with open("car_color_detector_app.py", "w") as f:
    f.write(code)

print("Created car_color_detector_app.py")

Created car_color_detector_app.py


In [6]:
!ls


car_color_detector_app.py  sample_data


In [7]:
!pip install pyngrok==4.1.1 streamlit
from pyngrok import ngrok

# Kill any previous tunnels
ngrok.kill()

# Start tunnel
public_url = ngrok.connect(port=8501)
public_url


  Preparing metadata (setup.py) ... done
  Created wheel for pyngrok: filename=pyngrok-4.1.1-py3-none-any.whl size=15963 sha256=9bc53ff4270b743901df56fabb0f78c4733f73d2e23c83d3f952c9ddf3ae7789
  Stored in directory: /root/.cache/pip/wheels/0d/1f/e9/3ce954f5c5d9d30ec279ad8bc5d44666a64fb4be782cb39a2d
Successfully built pyngrok


ERROR:pyngrok.process:t=2025-11-19T17:49:56+0000 lvl=eror msg="failed to auth" obj=tunnels.session err="Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n"



PyngrokNgrokError: The ngrok process errored on start: Usage of ngrok requires a verified account and authtoken.\n\nSign up for an account: https://dashboard.ngrok.com/signup\nInstall your authtoken: https://dashboard.ngrok.com/get-started/your-authtoken\r\n\r\nERR_NGROK_4018\r\n.

In [8]:
!pip install torchvision==0.15.2 torch==2.0.1


ERROR: Ignored the following yanked versions: 0.1.6, 0.1.7, 0.1.8, 0.1.9, 0.2.0, 0.2.1, 0.2.2, 0.2.2.post2, 0.2.2.post3
ERROR: Could not find a version that satisfies the requirement torchvision==0.15.2 (from versions: 0.17.0, 0.17.1, 0.17.2, 0.18.0, 0.18.1, 0.19.0, 0.19.1, 0.20.0, 0.20.1, 0.21.0, 0.22.0, 0.22.1, 0.23.0, 0.24.0, 0.24.1)
ERROR: No matching distribution found for torchvision==0.15.2


In [9]:
!pip install streamlit opencv-python torch torchvision
!streamlit run car_color_detector_app.py





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.28.0.12:8501
  External URL: http://34.124.219.76:8501

  Stopping...


In [1]:
import streamlit as st
import torch
import torchvision
from torchvision import transforms
import cv2
import numpy as np
from PIL import Image
import pandas as pd
import io
import os
from datetime import datetime
import pytz

In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
MODEL_CONFIDENCE_THRESHOLD = 0.5
TIMEZONE = 'Asia/Kolkata'

In [3]:
COCO_INSTANCE_CATEGORY_NAMES = [
'__background__', 'person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck',
'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog',
'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag',
'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove',
'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl',
'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair',
'couch', 'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard',
'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors',
'teddy bear', 'hair drier', 'toothbrush'
]
CAR_CLASS_NAMES = {'car', 'truck', 'bus', 'motorcycle'} # treat these as vehicles for counting

In [5]:
@st.cache_resource
def load_model():
    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
    model.eval()
    model.to(DEVICE)
    return model


model = load_model()

2025-11-19 17:55:13.271 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:55:13.390 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-11-19 17:55:13.391 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:55:13.392 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:55:13.393 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/torchvision/models/_utils.py:223: UserW

Downloading: "https://download.pytorch.org/models/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth" to /root/.cache/torch/hub/checkpoints/fasterrcnn_resnet50_fpn_coco-258fb6c6.pth


100%|██████████| 160M/160M [00:01<00:00, 120MB/s]
2025-11-19 17:55:16.196 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:55:16.197 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:55:16.200 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [9]:
def get_current_time_iso():
    tz = pytz.timezone(TIMEZONE)
    return datetime.now(tz).isoformat()


def detect_objects_pytorch(model, image: np.ndarray):
    # image is BGR (OpenCV). Convert to RGB PIL then to tensor
    img_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    pil = Image.fromarray(img_rgb)
    transform = transforms.Compose([transforms.ToTensor()])
    tensor = transform(pil).to(DEVICE)
    with torch.no_grad():
        outputs = model([tensor])
    return outputs[0]

In [10]:
def bbox_to_int(bbox):
    return [int(b) for b in bbox]


def crop_region(image, bbox):
    x1, y1, x2, y2 = bbox_to_int(bbox)
    h, w = image.shape[:2]
    x1 = max(0, x1); y1 = max(0, y1); x2 = min(w-1, x2); y2 = min(h-1, y2)
    return image[y1:y2, x1:x2]

In [12]:
def detect_dominant_color(bgr_image):
    # Convert to HSV and compute mean hue over non-black pixels
    if bgr_image.size == 0:
        return 'unknown', None
    hsv = cv2.cvtColor(bgr_image, cv2.COLOR_BGR2HSV)
    # Create mask for non-dark pixels
    v = hsv[:, :, 2]
    mask = v > 30
    if np.count_nonzero(mask) < 10:
        return 'unknown', None
    hues = hsv[:, :, 0][mask]
    mean_hue = np.mean(hues)
    # Hue in OpenCV: 0-179 (0 red, 60 yellow, 120 blue)
    # We'll classify blue if hue in range ~[90,130]
    if 90 <= mean_hue <= 130:
        return 'blue', mean_hue
    # simple other buckets (red, green, yellow) could be added; otherwise 'other'
    return 'other', mean_hue

In [14]:
def draw_bounding_boxes(frame, detections, labels_of_interest, conf_thresh=MODEL_CONFIDENCE_THRESHOLD):
    boxes = detections['boxes'].cpu().numpy()
    scores = detections['scores'].cpu().numpy()
    labels = detections['labels'].cpu().numpy()


    vehicles = []
    people_count = 0
    h, w = frame.shape[:2]


    for bbox, score, label_idx in zip(boxes, scores, labels):
        if score < conf_thresh:
            continue
        label_name = COCO_INSTANCE_CATEGORY_NAMES[label_idx]
        if label_name == 'person':
            people_count += 1
        if label_name in labels_of_interest:
            # Handle vehicle
            crop = crop_region(frame, bbox)
            color_label, hue = detect_dominant_color(crop)
            # According to requirement:
            # - show red rectangle for blue cars
            # - show blue rectangles for other colour cars
            if color_label == 'blue':
                rect_color = (0, 0, 255) # BGR red
            elif color_label == 'unknown':
                rect_color = (255, 0, 0) # blue as fallback
            else:
                rect_color = (255, 0, 0) # BGR blue
            x1, y1, x2, y2 = bbox_to_int(bbox)
            cv2.rectangle(frame, (x1, y1), (x2, y2), rect_color, 2)
            label_text = f"{label_name} ({color_label}) {score:.2f}"
            cv2.putText(frame, label_text, (x1, max(15, y1-6)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, rect_color, 1)
            vehicles.append({'bbox': bbox_to_int(bbox), 'score': float(score), 'label': label_name, 'color': color_label})


    return frame, vehicles, people_count

In [15]:
image_log_columns = ['timestamp', 'filename', 'num_vehicles', 'vehicle_details', 'num_people']
video_log_columns = ['timestamp', 'frame_index', 'num_vehicles', 'vehicle_details', 'num_people']



In [17]:
if 'image_log' not in st.session_state:
    st.session_state['image_log'] = pd.DataFrame(columns=image_log_columns)
if 'video_log' not in st.session_state:
    st.session_state['video_log'] = pd.DataFrame(columns=video_log_columns)

2025-11-19 17:58:47.227 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:58:47.230 Session state does not function when running a script without `streamlit run`
2025-11-19 17:58:47.241 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:58:47.242 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:58:47.244 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:58:47.245 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:58:47.247 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


In [18]:
st.set_page_config(page_title='Car Colour Detector', layout='wide')
st.title('Car Colour Detection & People Counter')

2025-11-19 17:59:02.183 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:59:02.187 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:59:02.189 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 17:59:02.191 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [21]:
from pyngrok import ngrok
ngrok.set_auth_token('YOUR_AUTHTOKEN')


In [23]:
col1, col2 = st.columns([1, 2])

with col1:
    st.header('Settings')
    st.write('Model: torchvision Faster R-CNN (pretrained on COCO)')
    st.write('Vehicle classes considered: car, truck, bus, motorcycle')
    st.write(f'Detection confidence threshold: {MODEL_CONFIDENCE_THRESHOLD}')
    st.markdown('---')
    st.header('Logs')
    if st.button('Download image log (.csv)'):
        csv = st.session_state['image_log'].to_csv(index=False)
        st.download_button('Download image log', csv, file_name='image_log.csv')
    if st.button('Download video log (.csv)'):
        csv = st.session_state['video_log'].to_csv(index=False)
        st.download_button('Download video log', csv, file_name='video_log.csv')

with col2:
    st.header('Upload an image')
    uploaded_file = st.file_uploader('Choose an image', type=['png', 'jpg', 'jpeg'])
    if uploaded_file is not None:
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        img = cv2.imdecode(file_bytes, cv2.IMREAD_COLOR)
        st.image(cv2.cvtColor(img, cv2.COLOR_BGR2RGB), caption='Uploaded image', use_column_width=True)
        with st.spinner('Detecting...'):
            detections = detect_objects_pytorch(model, img)
            out_frame, vehicles, people_count = draw_bounding_boxes(img.copy(), detections, CAR_CLASS_NAMES)
            st.image(cv2.cvtColor(out_frame, cv2.COLOR_BGR2RGB), caption='Detections', use_column_width=True)
        # Log results
        # Using pd.concat for appending, as .append is deprecated
        new_image_log_entry = pd.DataFrame([{
            'timestamp': get_current_time_iso(),
            'filename': uploaded_file.name,
            'num_vehicles': len(vehicles),
            'vehicle_details': vehicles,
            'num_people': people_count
        }], columns=image_log_columns)
        st.session_state['image_log'] = pd.concat([st.session_state['image_log'], new_image_log_entry], ignore_index=True)
        st.success(f'Found {len(vehicles)} vehicles and {people_count} people.')

    st.markdown('---')
    st.header('Real-time webcam (press Start)')
    start_webcam = st.checkbox('Start webcam')
    conf_thresh = st.slider('Confidence threshold', 0.1, 0.9, 0.5)

2025-11-19 18:02:29.373 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:02:29.377 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:02:29.377 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:02:29.379 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:02:29.381 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:02:29.382 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:02:29.384 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:02:29.386 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [25]:
if start_webcam:
    FRAME_WINDOW = st.image([])
    cap = cv2.VideoCapture(0)
    frame_idx = 0
    if not cap.isOpened():
        st.error('Webcam not available')
    else:
        try:
            while start_webcam:
                ret, frame = cap.read()
                if not ret:
                    st.error('Failed to read from webcam')
                    break
                detections = detect_objects_pytorch(model, frame)
                out_frame, vehicles, people_count = draw_bounding_boxes(frame.copy(), detections, CAR_CLASS_NAMES, conf_thresh)
                FRAME_WINDOW.image(cv2.cvtColor(out_frame, cv2.COLOR_BGR2RGB))
                # Log every Nth frame to avoid huge logs
                if frame_idx % 10 == 0 and (len(vehicles) > 0 or people_count > 0):
                    new_video_log_entry = pd.DataFrame([{
                        'timestamp': get_current_time_iso(),
                        'frame_index': frame_idx,
                        'num_vehicles': len(vehicles),
                        'vehicle_details': vehicles,
                        'num_people': people_count
                    }], columns=video_log_columns)
                    st.session_state['video_log'] = pd.concat([st.session_state['video_log'], new_video_log_entry], ignore_index=True)
                frame_idx += 1
                # Streamlit reruns the script on widget interaction.
                # No need to re-check inside the loop. The loop will naturally break if user unchecks and script reruns.
        finally:
            cap.release()

st.markdown('---')
st.write('Notes and tips:')
st.write('- For better car-color detection, use close, high-resolution images of vehicles.')
st.write('- The colour heuristic uses mean hue and is intentionally simple: you can replace it with k-means on the crop for improved robustness.')

st.write('\nIf you want, I can:')
st.write('- Provide a Colab-friendly version (with ngrok) so you can run the Streamlit UI from Colab.')
st.write('- Replace the simple colour heuristic with K-means clustering for dominant colour extraction.')
st.write('- Save detected car crop images alongside logs (zipped) for manual review.')

# End of file

2025-11-19 18:03:46.324 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:03:46.326 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:03:46.328 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:03:46.330 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:03:46.332 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:03:46.333 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:03:46.335 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-11-19 18:03:46.336 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar